In [22]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
import ngsolve
from xfem import *

In [23]:
from ngsolve import *
from math import pi

# Analytische Geschwindigkeit (als GridFunction oder CoefficientFunction)
uexact = CoefficientFunction((
    sin(pi*x) * cos(pi*y),   # u(x,y)
    -cos(pi*x) * sin(pi*y)   # v(x,y)
))
pexact = sin(pi*x) * sin(pi*y)
# Laplace von uexact (Komponentenweise)
lapu_x = -pi**2 * sin(pi*x) * cos(pi*y) * 2
lapu_y = -pi**2 * (-cos(pi*x) * sin(pi*y)) * 2  # Minuszeichen kommt von v

# Gradient von p
dp_dx = pi * cos(pi*x) * sin(pi*y)
dp_dy = pi * sin(pi*x) * cos(pi*y)

# Gesamte rechte Seite f = -Δu + ∇p
f = CoefficientFunction((
    -lapu_x + dp_dx,
    -lapu_y + dp_dy
))
r = sqrt(x**2 + y**2)
levelset = r - 1
#f= CF((0,x))

#uexact = CoefficientFunction((    -r * (1 - r**2) * y, r * (1 - r**2) * x))

# Analytischer Druck
#pexact = (r + 3 - 15 * r**2) * y


In [24]:
for maxh in [0.4,0.2]:
    for order in [1, 2, 3]:
        # Stabilization parameter for ghost-penalty
        gamma_stab = 100
        # Stabilization parameter for Nitsche
        lambda_nitsche = 10 * order * order

        beta0 =1
        beta1 = 1
        beta2 = 1
        beta3 = 1
        square = SplineGeometry()
        square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
        ngmesh = square.GenerateMesh(maxh=maxh)
        mesh = Mesh(ngmesh)


        lsetp1 = GridFunction(H1(mesh,order=1,autoupdate=True),autoupdate=True)
        print(type(lsetp1),type(levelset))
        #lsetp1.Set(levelset)

        InterpolateToP1(levelset,lsetp1)# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
        Draw(lsetp1, mesh, f"levelset_maxh{maxh}")
        ci = CutInfo(mesh, lsetp1)
        hasneg = ci.GetElementsOfType(HASNEG)
        neg = ci.GetElementsOfType(NEG)
        hasif = ci.GetElementsOfType(IF)
        haspos = ci.GetElementsOfType(HASPOS)
        ba_facets = GetFacetsWithNeighborTypes(mesh, a=haspos, b=any)
        patch_elements = ci.GetElementsOfType(ANY)
        #cut_elements = ci.GetElementsOfType(HASIF)  


        interior_facets = GetFacetsWithNeighborTypes(mesh, a=neg, b=neg)
        interface_facet_set = GetFacetsWithNeighborTypes(mesh, a=hasif, b=hasneg)
        

        
        h = specialcf.mesh_size
        n = Normalize(grad(lsetp1))

        # integration domains:
        dx = dCut(lsetp1, NEG, definedonelements=hasneg)
        ds = dCut(lsetp1, IF, definedonelements=hasif)

        dw = dFacetPatch(definedonelements=ba_facets)
        dw_int = dFacetPatch(definedonelements=interior_facets)
        dw_interface = dFacetPatch(definedonelements=interface_facet_set)
        V = VectorH1(mesh, order=order,dgjumps=True)
        V = Compress(V, GetDofsOfElements(V,hasneg))
        Q = H1(mesh, order=order-1)
        Q = Compress(Q, GetDofsOfElements(Q,hasneg))
        Z = NumberSpace(mesh)
        X = V*Q*Z
        (u,p,z),(v,q,y) = X.TnT()
        gfu = GridFunction(X)

        def jump(f):
            return f - f.Other()
        def jumpn(f):
            n = Normalize(grad(lsetp1))
            return Grad(f)*n - Grad(f).Other()*n

        a = BilinearForm(X)
        stokes = InnerProduct(Grad(u), Grad(v))*dx + div(u)*q*dx + div(v)*p*dx
        a += stokes
        a += -(Grad(u)*n * v + Grad(v)*n * u) * ds + gamma_stab / h * u * v * ds #nitzshe stabilization
        a += -(q*n * u + p*n * v) * ds
        a += p*y *dx + q *z*dx
        a += beta2* InnerProduct(jump(Grad(u)), jump(Grad(v))) * dw_interface #velocity ghost penalty
        a += -beta0 * InnerProduct(jump(p), jump(q)) * dw_interface #velocity ghost penalty
        #a += beta2/h* InnerProduct(Grad(u), Grad(v)) * ds #velocity ghost penalty
        #a += SymbolicFacetBFI(beta2 / h * InnerProduct(jump(grad(u)), jump(grad(v)))*dx,element_boundary=True,definedon=interface_facet_set)
        #a_vel = BilinearForm(V)
        #a_vel += beta2 / h * InnerProduct(Grad(u), Grad(v)) * dw_interface
        #a_vel.Assemble()

        # Dann separat zur globalen Matrix addieren:
        a.Assemble()
        #a.mat += a_vel.mat

        rhs = LinearForm(X)  # oder f*v*dx mit f gegeben
        rhs += InnerProduct(f, v)* dx
        rhs += -(Grad(v)*n * uexact) * ds + gamma_stab / h * uexact * v * ds
        rhs.Assemble()

        gfu.vec.data = a.mat.Inverse(X.FreeDofs()) * rhs.vec
        error_u = sqrt(Integrate( (gfu.components[0] - uexact) ** 2, mesh ))
        print("||u_h - u_exact||_L2 =", error_u)
        error_p = sqrt(Integrate( (gfu.components[1] - pexact) ** 2, mesh ))
        print("||u_h - p_exact||_L2 =", error_p ,"order=", order, "maxh=", maxh)

        #l2error.append(sqrt(Integrate((gfu - exact)**2*dx, mesh)))
        #Draw(gfu.components[0], mesh, "u")
        #Draw(gfu.components[1], mesh, "p")

        DrawDC(lsetp1,gfu.components[0], CF((0,0)), mesh, "u")
        DrawDC(lsetp1,gfu.components[1], 0, mesh, "p")



<class 'ngsolve.comp.GridFunction'> <class 'ngsolve.fem.CoefficientFunction'>
||u_h - u_exact||_L2 = 0.953766410856825
||u_h - p_exact||_L2 = 5.331978459916227 order= 1 maxh= 0.4
webgui discontinuous vis only for scalar functions a.t.m., switching to IfPos variant


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

<class 'ngsolve.comp.GridFunction'> <class 'ngsolve.fem.CoefficientFunction'>
||u_h - u_exact||_L2 = 0.9305958599032723
||u_h - p_exact||_L2 = 5.812329923085734 order= 2 maxh= 0.4
webgui discontinuous vis only for scalar functions a.t.m., switching to IfPos variant


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

...

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

<class 'ngsolve.comp.GridFunction'> <class 'ngsolve.fem.CoefficientFunction'>
||u_h - u_exact||_L2 = 2.3136904672968384
||u_h - p_exact||_L2 = 39.8929194084228 order= 3 maxh= 0.2
webgui discontinuous vis only for scalar functions a.t.m., switching to IfPos variant


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…